In [3]:
import requests
import json
import re
from urllib.parse import urlparse

class PolymarketPriceFetcher:
    def __init__(self):
        self.gamma_api_base = "https://gamma-api.polymarket.com"
        self.clob_api_base = "https://clob.polymarket.com"
        
    def extract_market_slug_from_url(self, url):
        """
        Extract market slug from Polymarket URL
        Example: https://polymarket.com/sports/mlb/games/week/11/mlb-sd-mil-2025-06-08
        Returns: mlb-sd-mil-2025-06-08
        """
        # Parse the URL path
        parsed = urlparse(url)
        path_parts = parsed.path.strip('/').split('/')
        
        # For sports URLs, the market slug is typically the last part
        if 'sports' in path_parts:
            return path_parts[-1]
        
        # For general market URLs like /market/slug-name
        if 'market' in path_parts:
            market_index = path_parts.index('market')
            if market_index + 1 < len(path_parts):
                return path_parts[market_index + 1]
        
        # Fallback: return the last part of the path
        return path_parts[-1] if path_parts else None

    def get_market_by_slug(self, market_slug):
        """
        Get market information using the Gamma API
        """
        try:
            # Search for markets with the specific slug
            url = f"{self.gamma_api_base}/markets"
            params = {
                'slug': market_slug,
                'active': True,
                'closed': False
            }
            
            response = requests.get(url, params=params)
            response.raise_for_status()
            
            markets = response.json()
            
            if not markets:
                print(f"No active market found for slug: {market_slug}")
                return None
                
            return markets[0]  # Return the first matching market
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching market data: {e}")
            return None

    def get_market_prices(self, market):
        """
        Get current prices for a market using the CLOB API
        """
        if not market or 'tokens' not in market:
            print(market)
            print("Invalid market data")
            return None
            
        try:
            prices = {}
            
            for token in market['tokens']:
                token_id = token['token_id']
                outcome = token['outcome']
                
                # Get the current price/midpoint for this token
                price_url = f"{self.clob_api_base}/midpoint"
                params = {'token_id': token_id}
                
                response = requests.get(price_url, params=params)
                response.raise_for_status()
                
                price_data = response.json()
                
                # The midpoint represents the current price
                if 'midpoint' in price_data:
                    prices[outcome] = {
                        'price': float(price_data['midpoint']),
                        'probability': round(float(price_data['midpoint']) * 100, 2),  # Convert to percentage
                        'token_id': token_id
                    }
                else:
                    print(f"No midpoint data available for {outcome}")
                    
            return prices
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching price data: {e}")
            return None

    def get_order_book(self, token_id):
        """
        Get the order book for a specific token to see bid/ask spreads
        """
        try:
            url = f"{self.clob_api_base}/book"
            params = {'token_id': token_id}
            
            response = requests.get(url, params=params)
            response.raise_for_status()
            
            return response.json()
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching order book: {e}")
            return None

    def fetch_market_prices(self, polymarket_url):
        """
        Main method to fetch market prices from a Polymarket URL
        """
        print(f"Fetching market data from: {polymarket_url}")
        
        # Extract market slug from URL
        market_slug = self.extract_market_slug_from_url(polymarket_url)
        if not market_slug:
            print("Could not extract market slug from URL")
            return None
            
        print(f"Extracted market slug: {market_slug}")
        
        # Get market information
        market = self.get_market_by_slug(market_slug)
        if not market:
            return None
            
        print(f"Found market: {market.get('question', 'Unknown question')}")
        
        # Get current prices
        prices = self.get_market_prices(market)
        if not prices:
            return None
            
        # Display results
        self.display_results(market, prices)
        
        return {
            'market': market,
            'prices': prices
        }

    def display_results(self, market, prices):
        """
        Display market information and prices in a formatted way
        """
        print("\n" + "="*60)
        print(f"MARKET: {market.get('question', 'Unknown')}")
        print(f"Description: {market.get('description', 'No description')[:100]}...")
        print(f"Volume: ${market.get('volume', 0):,.2f}")
        print(f"Liquidity: ${market.get('liquidity', 0):,.2f}")
        print("="*60)
        
        print("\nCURRENT PRICES:")
        print("-" * 40)
        
        for outcome, data in prices.items():
            print(f"{outcome}:")
            print(f"  Price: ${data['price']:.3f}")
            print(f"  Implied Probability: {data['probability']:.2f}%")
            print(f"  Token ID: {data['token_id']}")
            print()

def main():
    # Example usage
    url = "https://polymarket.com/sports/mlb/games/week/11/mlb-sd-mil-2025-06-08"
    
    fetcher = PolymarketPriceFetcher()
    result = fetcher.fetch_market_prices(url)
    
    if result:
        print("\nScript completed successfully!")
        
        # You can also get order book data for more detailed information
        prices = result['prices']
        if prices:
            first_outcome = list(prices.keys())[0]
            token_id = prices[first_outcome]['token_id']
            
            print(f"\nOrder book for {first_outcome}:")
            order_book = fetcher.get_order_book(token_id)
            if order_book:
                print(json.dumps(order_book, indent=2))
    else:
        print("Failed to fetch market data")

if __name__ == "__main__":
    main()

Fetching market data from: https://polymarket.com/sports/mlb/games/week/11/mlb-sd-mil-2025-06-08
Extracted market slug: mlb-sd-mil-2025-06-08
Found market: Padres vs. Brewers
{'id': '550520', 'question': 'Padres vs. Brewers', 'conditionId': '0x768f84b52cb98d52c312c6836ba3e49cfffb9fd142b5e07ca130aadc178cd7f7', 'slug': 'mlb-sd-mil-2025-06-08', 'resolutionSource': 'https://www.mlb.com/', 'endDate': '2025-06-15T17:05:00Z', 'liquidity': '113900.5118', 'startDate': '2025-06-07T08:01:05.080625Z', 'fee': '20000000000000000', 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/Repetitive-markets/MLB.jpg', 'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/Repetitive-markets/MLB.jpg', 'description': "In the upcoming MLB game, scheduled for June 8 at 1:05PM ET:\nIf the San Diego Padres win, the market will resolve to “Padres”.\nIf the Milwaukee Brewers win, the market will resolve to “Brewers”.\nIf the game is postponed, this market will remain open until the game has been c